**Travel Planner AI**

This chatbot assists travelers in:
1. Understanding their travel preferences
2. Recommending top hotels based on budget and preferences
3. Suggesting day-wise itineraries with attractions and restaurants
4. Providing transportation options

System Architecture (5 Layers):
- Intent Clarity Layer: Gathers user requirements
- Intent Confirmation Layer: Validates captured preferences
- Destination Mapping Layer: Extracts and classifies travel options
- Information Extraction Layer: Matches user profile with available options
- Recommendation Layer: Presents hotels and itineraries

**PART 1: IMPORTS**

In [121]:
import os
import json
import re
import ast
import pandas as pd
from groq import Groq

from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
MODEL_NAME = "llama-3.3-70b-versatile"

client = Groq(api_key=groq_api_key)

**PART 2: DATA LOADING**

**Dataset Description**

The dataset used in this project consists of structured travel information for a limited number of U.S. cities.
Each city record includes details such as:

- Popular attractions
- Recommended restaurants
- Accommodation options
- Local highlights and activities

The dataset was provided in JSONL format, where each line represents a city’s information in JSON structure. Example entries include:
Rockford, Pensacola, Philadelphia, Miami, and others.

⚙️ Note:
The current dataset includes a limited set of U.S. cities. The system is modular and can easily be extended to include global data or custom sources such as APIs (e.g., Google Places, TripAdvisor, or OpenStreetMap).

In [122]:
def load_dataset(file_path):
    """
    load travel data from JSON file containinig multiple city records
    returns a structured dictionary with cities as key
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    #split data by newlines
    city_data ={}
    for line in content.strip().split('\n'):
        if line.strip():
            data = json.loads(line)
            #now extract city name
            city_name = None
            for key in data.keys():
                if 'Attractions in' in key:
                    city_name = key.replace('Attractions in', '').strip()
                    break
            if city_name:
                city_data[city_name] = data
    return city_data

In [123]:
def create_accomodation_dataframe(city_data, city_name):
    """create a pandas dataframe from accommodation data for a specific city"""
    key = f"Accommodations in {city_name}"
    if key in city_data[city_name]:
        df = pd.DataFrame(city_data[city_name][key])
        df['city'] = city_name
        return df
    return pd.DataFrame()

In [124]:
def create_attractions_dataframe(city_data, city_name):
    """create a pandas dataframe from attractions data for a specific city"""
    key = f"Attractions in {city_name}"
    if key in city_data[city_name]:
        df = pd.DataFrame(city_data[city_name][key])
        df['city'] = city_name
        return df
    return pd.DataFrame()

In [125]:
def create_restaurants_dataframe(city_data, city_name):
    """create a pandas dataframe from restaurants data for a specific city"""
    key = f"Restaurants in {city_name}"
    if key in city_data[city_name]:
        df = pd.DataFrame(city_data[city_name][key])
        df['city'] = city_name
        return df
    return pd.DataFrame()

**PART 3: INTENT CLARITY AND CONFIRMATION LAYERS**

In [126]:
def initialize_conversation():
    """
    Initialize the conversation with system message and iinstructions
    returns a list with the system message
    """

    delimiter = "#####"

    example_user_req = {
        'destination': 'Denver',
        'trip_duration': '4',
        'accommodation_type': 'entire_home',
        'accommodation_budegt': 'medium',
        'dining_preference': 'medium',
        'attraction_preference': 'cultural and nature',
        'group_size': '2',
        'total_budget': '30000'
    }

    system_message = f"""
    You are an intelligent travel planning expert helping travelers plan the perfect trip.
    Ask relevant questions to fill this Python dictionary with traveler's profile:
    
    {{'destination': 'city name', 'trip_duration': 'number of days', 'accommodation_type': 'type', 
      'accommodation_budget': 'value', 'dining_preference': 'value', 'attraction_preference': 'value',
      'group_size': 'number', 'total_budget': 'amount'}}
    
    IMPORTANT RULES:
    - 'accommodation_budget' and 'dining_preference': 'low', 'medium', or 'high'
    - 'accommodation_type': 'entire home', 'private room', or 'any'
    - 'trip_duration', 'group_size', 'total_budget': numerical values
    - 'destination': exact city name
    - Do NOT randomly assign values - infer from user responses
    - When you have ALL information, output ONLY the dictionary in this exact format:
      {example_user_req}
    
    {delimiter}Sample Conversation:{delimiter}
    User: "Hi, I'm planning a trip to Denver."
    Assistant: "Wonderful! To help plan your trip, what's the purpose - leisure, business, or family vacation? How many days?"
    
    User: "Leisure trip with family for 3 days."
    Assistant: "Great! How many people? What accommodation type - entire home, private room, or flexible?"
    
    User: "4 people - 2 adults, 2 kids. We'd prefer an entire home."
    Assistant: "Perfect! What's your total budget for 3 days including accommodation, dining, activities? Dining preference - budget-friendly, mid-range, or fine dining?"
    
    User: "Total budget $2000. Mid-range restaurants."
    Assistant: "Excellent! What attractions interest you - museums/cultural sites, outdoor/nature, amusement parks, or mixed?"
    
    User: "Cultural sites and nature activities."
    Assistant: {example_user_req}
    {delimiter}
    
    Start with a warm welcome and ask about their destination and travel purpose.
    """
    
    return [{"role": "system", "content": system_message}]

In [127]:
def get_chat_model_completions(messages):
    """get completion from OpenAI ChatCompletion API"""
    client = Groq(api_key=groq_api_key)
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [128]:
def moderation_check(user_input):
    """custom moderation check using groq's llm"""
    prompt = f"""
    You are a content safety system.
    Analyze the following text and decide if it violates content guidelines.

    Categories to check:
    - hate speech or harassment
    - sexual or explicit content
    - violence or self-harm
    - criminal or unsafe instructions

    Input: {user_input}

    Reply only with one word:
    - "Flagged" if it violates any category
    - "Not Flagged" if it's safe
    
    """
    
    client = Groq(api_key=groq_api_key)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role":"user", "content": prompt}],
        temperature=0
    )

    result = response.choices[0].message.content.strip()
    return result

In [129]:
def intent_confirmation_layer(response_assistant):
    """ check if the assistant has captured all necessary travel information """
    
    delimiter = "#####"
    prompt = f"""

    You are a senior travel planning evaluator with an eye for detail.
    You are provided an input that should contain a traveler's complete profile

    Check if the input has these keys with proper values:
    - 'destination': should be a city name
    - 'trip_duration': should be a number
    - 'accommodation_type': should be 'entire home','private room', or 'any'
    - 'accommodation_budget': should be 'low', 'medium', or 'high'
    - 'dinning_preference': should be 'low', 'medium', 'high'
    - 'attraction_peference': should describe types of attractions
    - 'group_size': should be a number
    - 'total_budget': should be anumerical amount

    Output 'Yes' if the input contains a dictionary with all the keys properly filled. 
    Output 'No' if any key is missing or has invallid values

    Input: {response_assistant}
    
    Only output one word - Yes or No. 

    """ 
    confirmation = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are an evaluation assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return confirmation.choices[0].message.content.strip()


def dictionary_present(response):
    """ Extract the user profile dictionary from assistant's response"""
    delimiter = "#####"
    user_req = {
        'destination': 'Denver',
        'trip_duration': '4',
        'accommodation_type': 'entire_home',
        'accommodation_budget': 'medium',
        'dining_preference': 'medium',
        'attraction_preference': 'cultural and nature',
        'group_size': '3',
        'total_budget': '30000'
    }

    prompt = f"""You are a python expert. Extract the python dictionary from the input string. 
    The dictionary should match this format: {user_req}
    Extract the exact keys and values present in the input. 
    Output only the dictionary, nothing else.

    
    Input: {response}
    """

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "system", "content": prompt}]
    )
    return response.choices[0].message.content

In [130]:
def extract_dict_from_string(string):
    """ Extract dictionary from a string using regex"""
    regex_pattern = r"\{[^{}]+\}"
    dic_matches = re.findall(regex_pattern, string)

    if dic_matches:
        dic_string = dic_matches[0]
        try:
            dic = ast.literal_eval(dic_string)
        except (ValueError, SyntaxError):
            try:
                dic_string = dic_string.replace("'", '"')
                dic = json.loads(dic_string)
            except json.JSONDecodeError as e:
                print(f"[ERROR] Failed to parse: {dic_string}")
                print(f"[Error] {e}")
                return None
            
        dic = {k.lower(): str(v) for k, v in dic.items()}
        return dic
    print("[ERROR] No dictionary found in response")
    return None

**PART 4: DESTINATION MAPPING AND INFORMATION EXTRACTION LAYER**

In [131]:
def map_accommodation_features(accommodation_data, user_budget):
    """ 
    Map accommodation features based on user requirements
    Returns scored accommodations
    """

    accommodations = []
    budget_mapping = {
        'low': (0, 300), #per night rate in USD
        'medium': (300, 700),
        'high': (700, 2000)
    }

    budget_range = budget_mapping.get(user_budget.lower(), (0,2000))
    print(f"[DEBUG] Budget range for '{user_budget}': ${budget_range[0]}-${budget_range[1]} / night")
    
    for _, acc in accommodation_data.iterrows():
        price = acc.get('price', 0)
        if pd.isna(price):
            continue

        if budget_range[0] <= price <= budget_range[1]:
            
            #score based on review rating
            review_rate = acc.get('review rate number', 0)
            if pd.isna(review_rate):
                review_rate = 0

            #score based on max occupancy matching group size
            max_occupancy = acc.get('maximum occupancy', 0)
            if pd.isna(max_occupancy):
                max_occupancy = 0

            accommodations.append({
                'name': acc.get('NAME', 'Unknown'),
                'price': price,
                'room_type': acc.get('room_type', 'Unknown'),
                'house_rules': acc.get('house_rules', 'Unknown'),
                'minimum_nights': acc.get('minimum_nights', 1),
                'maximum_occupancy': int(max_occupancy),
                'review_rating': review_rate,
                'score': review_rate
            })
    print(f"[DEBUG] Found {len(accommodations)} accommodations in budget range")
    return sorted(accommodations, key=lambda x: x['score'], reverse=True)


In [132]:
def map_restaurants(restaurant_data, dining_preference):
    """
    Map restaurants based on dining preference
    """
    cost_mapping = {
        'low': (0, 30),
        'medium': (30, 70),
        'high': (70, 200)
    }
    
    cost_range = cost_mapping.get(dining_preference.lower(), (0, 200))
    
    restaurants = []
    for _, rest in restaurant_data.iterrows():
        avg_cost = rest.get('Average Cost', 0)
        if pd.isna(avg_cost):
            continue
            
        if cost_range[0] <= avg_cost <= cost_range[1]:
            rating = rest.get('Aggregate Rating', 0)
            if pd.isna(rating):
                rating = 0
                
            restaurants.append({
                'name': rest.get('Name', 'Unknown'),
                'average_cost': avg_cost,
                'cuisines': rest.get('Cuisines', 'Various'),
                'rating': rating
            })
    
    return sorted(restaurants, key=lambda x: x['rating'], reverse=True)


def map_attractions(attraction_data, preference):
    """
    Map attractions based on user preference
    Filter attractions that match user's interests
    """
    attractions = []
    
    # Keywords for different attraction typees
    cultural_keywords = ['museum', 'historic', 'art', 'gallery', 'heritage', 'cultural']
    nature_keywords = ['park', 'garden', 'nature', 'botanical', 'zoo', 'outdoor']
    adventure_keywords = ['adventure', 'sports', 'activities', 'thrill']
    family_keywords = ['children', 'family', 'kids', 'amusement']
    
    preference_lower = preference.lower()
    
    for _, attr in attraction_data.iterrows():
        name = attr.get('Name', '').lower()
        score = 0
        
        # scorebased on preference match
        if any(keyword in preference_lower for keyword in ['cultural', 'museum', 'history']):
            if any(keyword in name for keyword in cultural_keywords):
                score += 2
        
        if any(keyword in preference_lower for keyword in ['nature', 'outdoor', 'park']):
            if any(keyword in name for keyword in nature_keywords):
                score += 2
        
        if any(keyword in preference_lower for keyword in ['adventure', 'active']):
            if any(keyword in name for keyword in adventure_keywords):
                score += 2
                
        if any(keyword in preference_lower for keyword in ['family', 'kids', 'children']):
            if any(keyword in name for keyword in family_keywords):
                score += 2
        
        # include all attractions
        attractions.append({
            'name': attr.get('Name', 'Unknown'),
            'address': attr.get('Address', 'N/A'),
            'phone': attr.get('Phone', 'N/A'),
            'website': attr.get('Website', 'N/A'),
            'score': score if score > 0 else 1
        })
    
    return sorted(attractions, key=lambda x: x['score'], reverse=True)


In [133]:
def generate_itinerary(attractions, restaurants, trip_duration):
    """ Generate a day-wise itinerary"""
    days = int(trip_duration)
    itinerary = []

    attractions_per_day = max(2, len(attractions) // days)
    
    for day in range(1, days + 1):
        start_idx = (day - 1) * attractions_per_day
        end_idx = start_idx + attractions_per_day

        day_attractions = attractions[start_idx:end_idx]
        day_restaurants = restaurants[(day-1)*2: day*2] if len(restaurants) > (day-1)*2 else restaurants[:2]

        itinerary.append({
            'day': day,
            'attractions': day_attractions,
            'restaurants': day_restaurants
        })
    return itinerary

In [134]:
def compare_travel_options_with_user(user_req_string, city_data):
    """main fucntion to match user requirements with available travel options
    return top accommodations and itinerary
    """

    user_requirements = extract_dict_from_string(user_req_string)
    
    destination = user_requirements.get('destination', '').strip()
    trip_duration = user_requirements.get('trip_duration', '3')
    accommodation_budget = user_requirements.get('accommodation_budget', 'medium')
    dinning_preference = user_requirements.get('dinning_preference', 'medium')
    attraction_preference = user_requirements.get('attraction_preference', 'all')
    group_size = int(user_requirements.get('group_size', '2'))

    print(f"\n[DEBUG] Looking for destination: '{destination}'")
    print(f"[DEBUG] Available cities: {list(city_data.keys())}...")
    
    matched_city = None
    if destination in city_data:
        matched_city = destination
    else:
        for city in city_data.keys():
            if city.lower() == destination.lower():
                matched_city = city
                print(f"[DEBUG] Matched '{destination}' to '{city}'")
                break
    
    if matched_city is None:
        print(f"[DEBUG] City '{destination}' not Found!")
        return None
    
    destination = matched_city
    
    accommodation_df = create_accomodation_dataframe(city_data, destination)
    restaurant_df = create_restaurants_dataframe(city_data, destination)
    attraction_df = create_attractions_dataframe(city_data, destination)

    accommodations = map_accommodation_features(accommodation_df, accommodation_budget)
    #now filter accommodation by group size
    
    size_accommodations = [
        acc for acc in accommodations if acc['maximum_occupancy'] >= group_size
    ][:3]

    restaurants = map_restaurants(restaurant_df, dinning_preference)[:10]
    attractions = map_attractions(attraction_df, attraction_preference)[:15]


    #genrate itinerary
    itinerary = generate_itinerary(attractions, restaurants, trip_duration)

    return {
        'accommodations': size_accommodations,
        'itinerary': itinerary,
        'destination': destination
    }

**PART 5: RECCOMMENDATION LAYER**

In [135]:
def initialize_conv_reco(recommendations):
    """ Initializw conversation for recommendations """

    accommodations = recommendations['accommodations']
    itinerary = recommendations['itinerary']
    destination = recommendations['destination']

    acc_text = "\n".join([
        f"{i+1}. {acc['name']}: ${acc['price']}/night, {acc['room_type']}, "
        f"Sleeps {acc['maximum_occupancy']}, Rating: {acc['review_rating']}/5, "
        f"Min Stay: {acc['minimum_nights']} nights"
        for i, acc in enumerate(accommodations)
    ])
    
    # Format itinerary
    itinerary_text = ""
    for day_plan in itinerary:
        day_num = day_plan['day']
        itinerary_text += f"\n\nDay {day_num}:\n"
        itinerary_text += "Attractions:\n"
        for attr in day_plan['attractions'][:3]:
            itinerary_text += f"  - {attr['name']}\n"
        itinerary_text += "Dining Options:\n"
        for rest in day_plan['restaurants'][:2]:
            itinerary_text += f"  - {rest['name']} ({rest['cuisines']}), ~${rest['average_cost']} per person\n"
    
    system_message = f"""
    You are an intelligent travel planning expert helping travelers with their trip to {destination}.
    
    Based on the traveler's preferences, here are the recommendations:
    
    TOP ACCOMMODATIONS:
    {acc_text}
    
    SUGGESTED ITINERARY:
    {itinerary_text}
    
    Your role is to:
    1. Present these recommendations in a friendly, conversational manner
    2. Answer questions about accommodations, attractions, or restaurants
    3. Help travelers make decisions based on their specific needs
    4. Provide additional details or alternatives if requested
    
    Start by presenting the top accommodations and then the day-wise itinerary.
    Format your response in a clear, easy-to-read manner.
    """
    
    conversation = [{"role": "system", "content": system_message}]
    return conversation


**PART 6: DIALOGUE MANAGEMENT SYSTEM**

In [136]:
def dialogue_mgmt_system(city_data):
    
    """ Main dialogue managment system that orchestrates the conversation"""
    
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(" ASSISTANT:", introduction, "\n")

    recommendations = None 
    user_input = ''
    conversation_recom = None

    while user_input.lower() != "exit":
        user_input = input("YOU: ")
        print()

        if user_input.lower() == "exit":
            print("ASSISTANT: Thank you for using Travel Planner AI!")
            break

        moderation = moderation_check(user_input)
        if moderation == "Flagged":
            print("ASSISTANT: Sorry, this message has been flagged. Please enter valid user request. \n")
            continue

        if recommendations is None:
            conversation.append({"role": "user", "content": user_input})
            response_assistant = get_chat_model_completions(conversation)

            #moderationnc check on assisatant response
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("ASSISTANT: Sorry, I need to rephrase my response")
                continue

            #check requirements are complete
            confirmation = intent_confirmation_layer(response_assistant)

            if 'No' in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("ASSISTANT:", response_assistant, "\n")
            else:
                print("ASSISTANT:", response_assistant, "\n")

                response_dict = dictionary_present(response_assistant)
                print("\n[System: Processing your requirements...]\n")

                #get recommendations
                recommendations = compare_travel_options_with_user(response_dict, city_data)
                if recommendations is None or len(recommendations['accommodations']) == 0:
                    print("ASSISTANT: I apologize, but I coundn't find suitable options for your requirements.")
                    print("This might be because:")
                    print("- No accommodations match your budget and group size")
                    print("\nWould you like to try a different destination or adjust your preference")
                    recommendations = None
                    continue

                #initialize recommendation conversation
                conversation_recom = initialize_conv_reco(recommendations)
                recommendation_response = get_chat_model_completions(conversation_recom)
                
                conversation_recom.append({"role": "user", "content": "show me the recommendations"})
                conversation_recom.append({"role": "assistant", "content": recommendation_response})

                print("ASSISTANT:", recommendation_response, "\n")
            
        else: 
            #already have recommendations
            conversation_recom.append({"role": "user", "content": user_input})
            response_recom = get_chat_model_completions(conversation_recom)

            moderation = moderation_check(response_recom)
            if moderation == "Flagged":
                print("ASSISTANT: Sorry, I can not process that request.\n")
                continue
            print("ASSISTANT:", response_recom, "\n")
            conversation_recom.append({"role":"assistant", "content":response_recom})

**PART 7: MAIN EXECUTION**

In [137]:
print("=" * 70)
print("TRAVEL PLANNER AI")
print("Your Intelligent Travel Planning Assistant")
print("=" * 70)
print("\nLoading travel data....")

city_data = load_dataset('train_ref_info.jsonl')

print("\n" + "=" * 70)
print("Ready to start planning your trip!")
print("Type 'exit' anytime to end the conversation")
print("=" * 70 + "\n")


dialogue_mgmt_system(city_data)

TRAVEL PLANNER AI
Your Intelligent Travel Planning Assistant

Loading travel data....

Ready to start planning your trip!
Type 'exit' anytime to end the conversation

 ASSISTANT: Welcome to our travel planning service! We're excited to help you plan an unforgettable trip. To get started, could you please tell us what city you're interested in visiting? Is it for leisure, business, or a family vacation? Additionally, what's the approximate duration of your trip? 


ASSISTANT: Abilene sounds like a great destination for a solo traveler like yourself. Given your high budget and preferences, I'd be happy to recommend some top-notch private rooms, hotels, and dining options for you.

For a high-budget private room, I'd suggest considering hotels like the Marriott or the Hilton, which offer luxurious rooms and excellent amenities. Alternatively, you could also look into high-end bed and breakfasts or boutique hotels that offer a more personalized experience.

For dining, Abilene has a range 